# 美联储理事会BOG 理事任期数据爬取

@author: WUBIN ZHANG / 01208663

@date: Oct 16, 2024

In [1]:
import os
import pickle
import sys
sys.path.append('../')
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from utils.file_saver import json_dump
from utils.common import parse_datestring
from datetime import datetime

today = datetime.now().strftime("%b. %d, %Y")

## 任期表数据爬取

In [2]:
def extract_table_data(table_element):
    rows = table_element.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all(["th", "td"])
        cols = [col.text.strip() for col in cols]
        data.append(cols)
    return data

def extract_tables():
    # 替换为你的ChromeDriver路径
    driver = webdriver.Chrome()

    # 访问目标网址
    url = "https://www.federalreserve.gov/aboutthefed/bios/board/boardmembership.htm"
    driver.get(url)
    time.sleep(3)  # 等待页面加载完成
    
    # 获取所有表格元素
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tables = soup.find_all('table')

    needed_tables = {}
    for table in tables:
        previous_h4 = table.find_previous("h4")
        if previous_h4 and previous_h4.text.startswith('Chairs and Active Executive Officers of the Board of Governors of the Federal Reserve'):
            needed_tables['ChairTable'] = table
        elif previous_h4 and previous_h4.text.startswith('Vice Chair of the Board of Governors of the Federal Reserve System'):
            needed_tables['ViceChairTable'] = table
        elif previous_h4 and previous_h4.text.startswith('Vice Chair for Supervision of the Board of Governors of the Federal Reserve System'):
            needed_tables['SupervisionViceChairTable'] = table
        elif previous_h4 and previous_h4.text.startswith(
            "Members of the Federal Reserve Board"
        ):
            needed_tables['GovernorsTable'] = table
        else:
            continue

    # 提取每个表格的数据
    chair_table = extract_table_data(needed_tables['ChairTable']) # tables[3]
    vice_chair_table = extract_table_data(needed_tables["ViceChairTable"])  # tables[5]
    supervision_vice_chair_table = extract_table_data(tables[7])  # tables[7]
    governors_table = extract_table_data(tables[8])  # tables[8]

    # 打印结果检查
    print("Chair Table:", chair_table)
    print("Vice Chair Table:", vice_chair_table)
    print("Supervision Vice Chair Table:", supervision_vice_chair_table)
    print("Governors Table:", governors_table)
    return chair_table, vice_chair_table, supervision_vice_chair_table, governors_table


FILENAME = "../data/bog/board_of_governors_terms.pkl"
if os.path.exists(FILENAME):
    print(f"{FILENAME} exists, skipping extraction. Loading...")
    with open(FILENAME, "rb") as f:
        chair_table, vice_chair_table, supervision_vice_chair_table, governors_table = pickle.load(f)
else:
    chair_table, vice_chair_table, supervision_vice_chair_table, governors_table = extract_tables()
    with open(FILENAME, "wb") as f:
        pickle.dump([chair_table, vice_chair_table, supervision_vice_chair_table, governors_table], f)

../data/bog/board_of_governors_terms.pkl exists, skipping extraction. Loading...


In [3]:
json_dump(chair_table, "../data/bog/chair_table.json")
json_dump(vice_chair_table, "../data/bog/vice_chair_table.json")
json_dump(supervision_vice_chair_table, "../data/bog/supervision_vice_chair_table.json")
json_dump(governors_table, "../data/bog/governors_table.json")

### 主席任期表

In [4]:
def amend_date(date: str):
    try:
        dt, year = date.split(",")
        if len(year.strip())>=4:
            year = year.strip()[:4]
        return dt + ', ' + year
    except ValueError:
        return None

def split_term(term_str: str):
    try:
        start_date, end_date = term_str.split("-", maxsplit=2)
        start_date = amend_date(start_date.strip())
        end_date = amend_date(end_date.strip())
        return pd.Series([start_date, end_date])
    except ValueError:
        return pd.Series([None, None])

split_term("Feb. 1, 19362 - Jan. 31, 1948")

0     Feb. 1, 1936
1    Jan. 31, 1948
dtype: object

In [5]:
chairs = pd.DataFrame(chair_table[1:], columns=chair_table[0])
# chairs[["begin_date", "end_date"]] = chairs["Date of term"].str.split(" - ", expand=True)
chairs[["TermBeginDate", "TermEndDate"]] = chairs["Date of term"].apply(split_term)
# 保存到本地
chairs.to_csv("../data/bog/chairs.csv", index=False)
chairs

,Date of term,Appointee,TermBeginDate,TermEndDate
0,"Feb. 1, 19362 - Jan. 31, 1948",Marriner S. Eccles,"Feb. 1, 1936","Jan. 31, 1948"
1,"Apr. 15, 1948 - Mar. 31, 1951",Thomas B. McCabe,"Apr. 15, 1948","Mar. 31, 1951"
2,"Apr. 2, 1951 - Jan. 31, 1970","Wm. McC. Martin, Jr","Apr. 2, 1951","Jan. 31, 1970"
3,"Feb. 1, 1970 - Jan. 31, 1978",Arthur F. Burns,"Feb. 1, 1970","Jan. 31, 1978"
4,"Mar. 8, 1978 - Aug. 6, 1979",G. William Miller,"Mar. 8, 1978","Aug. 6, 1979"
5,"Aug. 6, 1979 - Aug. 11, 1987",Paul A. Volcker,"Aug. 6, 1979","Aug. 11, 1987"
6,"Aug. 11, 1987 - Jan. 31, 2006",Alan Greenspan,"Aug. 11, 1987","Jan. 31, 2006"
7,"Feb. 1, 2006 - Jan. 31, 2014",Ben S. Bernanke,"Feb. 1, 2006","Jan. 31, 2014"
8,"Feb. 3, 2014 - Feb. 3, 2018",Janet L. Yellen,"Feb. 3, 2014","Feb. 3, 2018"
9,"Feb. 5, 2018 -",Jerome H. Powell,"Feb. 5, 2018",None


In [6]:
chairs_terms = chairs[["Appointee", "TermBeginDate", "TermEndDate"]].copy(deep=True)
chairs_terms.rename(columns={"Appointee": "Name"}, inplace=True)
chairs_terms["Officier"] = "Chair"
chairs_terms.to_csv("../data/bog/chairs_terms.csv", index=False)
chairs_terms

,Name,TermBeginDate,TermEndDate,Officier
0,Marriner S. Eccles,"Feb. 1, 1936","Jan. 31, 1948",Chair
1,Thomas B. McCabe,"Apr. 15, 1948","Mar. 31, 1951",Chair
2,"Wm. McC. Martin, Jr","Apr. 2, 1951","Jan. 31, 1970",Chair
3,Arthur F. Burns,"Feb. 1, 1970","Jan. 31, 1978",Chair
4,G. William Miller,"Mar. 8, 1978","Aug. 6, 1979",Chair
5,Paul A. Volcker,"Aug. 6, 1979","Aug. 11, 1987",Chair
6,Alan Greenspan,"Aug. 11, 1987","Jan. 31, 2006",Chair
7,Ben S. Bernanke,"Feb. 1, 2006","Jan. 31, 2014",Chair
8,Janet L. Yellen,"Feb. 3, 2014","Feb. 3, 2018",Chair
9,Jerome H. Powell,"Feb. 5, 2018",None,Chair


### 副主席任期表

In [7]:
vice_chairs = pd.DataFrame(vice_chair_table[1:], columns=vice_chair_table[0])
vice_chairs[["TermBeginDate", "TermEndDate"]] = vice_chairs["Date of term"].apply(
    split_term
)
# 保存到本地
vice_chairs.to_csv("../data/bog/vice_chairs.csv", index=False)
vice_chairs

,Date of term,Appointee,TermBeginDate,TermEndDate
0,"Aug. 6, 1936 - Dec. 2, 1947",Ronald Ransom,"Aug. 6, 1936","Dec. 2, 1947"
1,"Mar. 11, 1955 - Feb. 28, 1966",C. Canby Balderston,"Mar. 11, 1955","Feb. 28, 1966"
2,"Mar. 1, 1966 - Apr. 30, 1973",J.L. Robertson,"Mar. 1, 1966","Apr. 30, 1973"
3,"May 1, 1973 - Feb. 13, 1976",George W. Mitchell,"May 1, 1973","Feb. 13, 1976"
4,"Feb. 13, 1976 - Nov. 19, 1978",Stephen S. Gardner,"Feb. 13, 1976","Nov. 19, 1978"
5,"July 27, 1979 - Feb. 11, 1982",Frederick H. Schultz,"July 27, 1979","Feb. 11, 1982"
6,"Mar. 31, 1982 - Apr. 30, 1986",Preston Martin,"Mar. 31, 1982","Apr. 30, 1986"
7,"Aug. 4, 1986 - Aug. 3, 1990",Manuel H. Johnson,"Aug. 4, 1986","Aug. 3, 1990"
8,"July 24, 1991 - Feb. 14, 1994","David W. Mullins, Jr.","July 24, 1991","Feb. 14, 1994"
9,"June 27, 1994 - Jan. 31, 1996",Alan S. Blinder,"June 27, 1994","Jan. 31, 1996"


In [8]:
vice_chairs_terms = vice_chairs[["Appointee", "TermBeginDate", "TermEndDate"]].copy(
    deep=True
)
vice_chairs_terms.rename(columns={"Appointee": "Name"}, inplace=True)
vice_chairs_terms["Officier"] = "Vice Chair"
vice_chairs.to_csv("../data/bog/vice_chair_terms.csv", index=False)
vice_chairs_terms

,Name,TermBeginDate,TermEndDate,Officier
0,Ronald Ransom,"Aug. 6, 1936","Dec. 2, 1947",Vice Chair
1,C. Canby Balderston,"Mar. 11, 1955","Feb. 28, 1966",Vice Chair
2,J.L. Robertson,"Mar. 1, 1966","Apr. 30, 1973",Vice Chair
3,George W. Mitchell,"May 1, 1973","Feb. 13, 1976",Vice Chair
4,Stephen S. Gardner,"Feb. 13, 1976","Nov. 19, 1978",Vice Chair
5,Frederick H. Schultz,"July 27, 1979","Feb. 11, 1982",Vice Chair
6,Preston Martin,"Mar. 31, 1982","Apr. 30, 1986",Vice Chair
7,Manuel H. Johnson,"Aug. 4, 1986","Aug. 3, 1990",Vice Chair
8,"David W. Mullins, Jr.","July 24, 1991","Feb. 14, 1994",Vice Chair
9,Alan S. Blinder,"June 27, 1994","Jan. 31, 1996",Vice Chair


### 监管副主席任期表

In [9]:
supervision_vice_chairs = pd.DataFrame(
    supervision_vice_chair_table[1:], columns=supervision_vice_chair_table[0]
)
supervision_vice_chairs[["TermBeginDate", "TermEndDate"]] = supervision_vice_chairs[
    "Date of term"
].apply(split_term)
# 保存到本地
supervision_vice_chairs.to_csv("../data/bog/supervision_vice_chairs.csv", index=False)
supervision_vice_chairs

,Date of term,Appointee,TermBeginDate,TermEndDate
0,"Oct. 13, 2017 - Oct. 13, 2021",Randal K. Quarles,"Oct. 13, 2017","Oct. 13, 2021"
1,"July 19, 2022 -",Michael S. Barr,"July 19, 2022",None


In [10]:
supervision_vice_chairs_terms = supervision_vice_chairs[
    ["Appointee", "TermBeginDate", "TermEndDate"]
].copy(deep=True)
supervision_vice_chairs_terms.rename(columns={"Appointee": "Name"}, inplace=True)
supervision_vice_chairs_terms["Officier"] = "Vice Chair for Supervision"
# supervision_vice_chairs_terms["TermEndDate"] = supervision_vice_chairs_terms[
#     "TermEndDate"
# ].fillna(today)
supervision_vice_chairs.to_csv("../data/bog/supervision_vice_chair_terms.csv", index=False)
supervision_vice_chairs_terms

,Name,TermBeginDate,TermEndDate,Officier
0,Randal K. Quarles,"Oct. 13, 2017","Oct. 13, 2021",Vice Chair for Supervision
1,Michael S. Barr,"July 19, 2022",None,Vice Chair for Supervision


### 理事任期表

In [11]:
import re
"John Skelton Williams2".strip("0123456789 ")

'John Skelton Williams'

In [12]:
governors = pd.DataFrame(governors_table[1:], columns=governors_table[0])
# 分离Name and District
governors[["Name", "District"]] = governors["Name and District"].str.split('\n', expand=True)
governors["Name"] = governors["Name"].str.strip("0123456789 ")
governors["District"] = governors["District"].str.strip()
# 将日期转为 datetime
# governors["TermBeginDate"] = governors["Date of initial oath of office"].transform(
#     lambda x: parse_datestring(x)
# )
governors["TermBeginDate"] = governors["Date of initial oath of office"]
# 保存到本地
governors.to_csv("../data/bog/governors.csv", index=False)
governors

,Name and District,Date of initial oath of office,Other dates and information relating to membership,Name,District,TermBeginDate
0,W.G. McAdoo\n\t\t\t Secretary of the Treasury,"Dec. 23, 1913","Dec. 15, 1918",W.G. McAdoo,Secretary of the Treasury,"Dec. 23, 1913"
1,John Skelton Williams2\n\t\t\t Comptroller o...,"Feb. 2, 1914","Mar. 2, 1921",John Skelton Williams,Comptroller of the Currency,"Feb. 2, 1914"
2,Charles S. Hamlin\n\t\t\t Boston,"Aug. 10, 1914",Reappointed in 1916 and 1926. Served until Feb...,Charles S. Hamlin,Boston,"Aug. 10, 1914"
3,Paul M. Warburg\n\t\t\t New York,"Aug. 10, 1914","Term expired Aug. 9, 1918.",Paul M. Warburg,New York,"Aug. 10, 1914"
4,Frederic A. Delano\n\t\t\t Chicago,"Aug. 10, 1914","Resigned July 21, 1918.",Frederic A. Delano,Chicago,"Aug. 10, 1914"
...,...,...,...,...,...,...
108,Christopher J. Waller\n\t\t\t Minneapolis,"Dec. 18, 2020",,Christopher J. Waller,Minneapolis,"Dec. 18, 2020"
109,Lisa D. Cook\n\t\t\t Atlanta,"May 23, 2022","Reappointed Sept. 13, 2023.",Lisa D. Cook,Atlanta,"May 23, 2022"
110,Philip N. Jefferson\n\t\t\t New York,"May 23, 2022",,Philip N. Jefferson,New York,"May 23, 2022"
111,Michael S. Barr\n\t\t\t Chicago,"July 19, 2022",,Michael S. Barr,Chicago,"July 19, 2022"


In [13]:
governors.loc[governors["Name"] == "Janet L. Yellen"]

,Name and District,Date of initial oath of office,Other dates and information relating to membership,Name,District,TermBeginDate
86,Janet L. Yellen\n\t\t\t San Francisco,"Aug. 12, 1994","Resigned Feb. 17, 1997; reappointed Oct. 4, 20...",Janet L. Yellen,San Francisco,"Aug. 12, 1994"


### 任期结束时间处理

In [14]:
# 任期结束日期 
options = [
    "Resigned",
    "resigned",
    "Died",
    "Retired",
    "Served until",
    "Served through",
    "Term expired",
    "term expired",
]

term_end_pattern = (
    # r"(?<=" + "[" + "|".join(options) + "]" + r"\s)[A-Za-z]+.\s\d{1,2},\s\d{4}"
    r"(" + "|".join(options) + r")\s([A-Za-z]+.\s\d{1,2},\s\d{4})"
)
print(term_end_pattern)
end_date_pattern = r"^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec).\s+(\d{1,2}),\s+(\d{4})$"
print(end_date_pattern)


def match_resign_date(desc: str):
    """匹配纯粹的日期字符串

    Args:
        desc (str): _description_

    Returns:
        _type_: _description_
    """
    match = re.findall(end_date_pattern, desc)
    if match:
        return [desc]
    else:
        return []

def get_resigned_dates(desc: str):
    """获取任期结束日期

    Args:
        desc (_type_): _description_

    Returns:
        _type_: _description_
    """
    if desc is None or pd.isna(desc):
        return "No resigned match found"

    # 匹配辞职期末日期
    resign_date = match_resign_date(desc)
    if resign_date:
        return resign_date
    match = re.findall(term_end_pattern, desc)
    match = [x[1] for x in match]

    if match:
        return match
    else:
        return []

term_end_examples = [
    "Mar. 4, 1933",
    "Term expired Jan. 31, 1996",
    "term expired Jan. 31, 1996",
    "Served until Sept. 1, 1950",
    "Served through Sept. 1, 1950",
    "Died Oct. 21, 1996",
    "Retired Apr. 30, 1967",
    "Resigned Dec. 31, 1996",
    "resigned Dec. 31, 1996",
    "reappointed Aug. 9, 1918.",
    "dadfaddafmwqeasd Aug. 9, 1918.",
]
for example in term_end_examples:
    match = get_resigned_dates(example)
    print(match)

(Resigned|resigned|Died|Retired|Served until|Served through|Term expired|term expired)\s([A-Za-z]+.\s\d{1,2},\s\d{4})
^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec).\s+(\d{1,2}),\s+(\d{4})$
['Mar. 4, 1933']
['Jan. 31, 1996']
['Jan. 31, 1996']
['Sept. 1, 1950']
['Sept. 1, 1950']
['Oct. 21, 1996']
['Apr. 30, 1967']
['Dec. 31, 1996']
['Dec. 31, 1996']
[]
[]


In [15]:
def get_reappointed_dates(desc):
    if desc is None or pd.isna(desc): 
        return "No resigned match found"
    # 再次任期日期
    options = [
        "Reappointed",
        "reappointed",
    ]
    reappointed_pattern = (
        # r"(?<=" + "[" + "|".join(options) + "]" + r"\s)[A-Za-z]+.\s\d{1,2},\s\d{4}"
        r"(" + "|".join(options) + r")\s([A-Za-z]+.\s\d{1,2},\s\d{4})"
    )
    match = re.findall(reappointed_pattern, desc)
    match = [x[1] for x in match]
    if match:
        return match
    else:
        return []

reappoint_examples = [
    "Reappointed Aug. 9, 1918.",
    "reappointed Aug. 9, 1918.",
    "Term expired Aug. 9, 1918.",
    "Term1 expired Oct. 9, 1918.",
]
for example in reappoint_examples:
    match = get_reappointed_dates(example)
    print(match)

['Aug. 9, 1918']
['Aug. 9, 1918']
[]
[]


In [16]:
# 再任命和辞职信息登记.
resigns = []
reappoints = []

governor_term_info = {}
for index, row in governors.iterrows():
    if row["Other dates and information relating to membership"] != "":
        # 辞职记录
        resign_dates = get_resigned_dates(row["Other dates and information relating to membership"])
        for resign_date in resign_dates:
            res = row.to_dict()
            res.update({"TermEndDate": resign_date})
            resigns.append(res)
        # 再职记录
        reappoint_dates = get_reappointed_dates(row["Other dates and information relating to membership"])
        for reappoint_date in reappoint_dates:
            res = row.to_dict()
            res.update({"ReappointedDate": reappoint_date})
            reappoints.append(res)
        # 辞职、在职、辞职... 交替拆入队列
        term_info = {"resign_dates": resign_dates, "reappoint_dates": reappoint_dates}
        governor_term_info.update({row['Name']: term_info})
        
json_dump(governor_term_info, "../data/bog/governor_term_info.json")
# 新增辞职记录和再职记录若干行
resigned = pd.DataFrame(resigns)
reappointed = pd.DataFrame(reappoints)
# governors = pd.concat([governors, resigned, reappointed], axis=0)
# governors.loc[governors["Date"]>=pd.to_datetime("2010-01-01", format="%Y-%m-%d")]
# governors

In [17]:
print(f"resigned: {len(resigned)}")
print(f"reappointed: {len(reappointed)}")
print(f"total: {len(resigned)+len(reappointed)}")

resigned: 107
reappointed: 5
total: 112


In [18]:
resigned

,Name and District,Date of initial oath of office,Other dates and information relating to membership,Name,District,TermBeginDate,TermEndDate
0,W.G. McAdoo\n\t\t\t Secretary of the Treasury,"Dec. 23, 1913","Dec. 15, 1918",W.G. McAdoo,Secretary of the Treasury,"Dec. 23, 1913","Dec. 15, 1918"
1,John Skelton Williams2\n\t\t\t Comptroller o...,"Feb. 2, 1914","Mar. 2, 1921",John Skelton Williams,Comptroller of the Currency,"Feb. 2, 1914","Mar. 2, 1921"
2,Paul M. Warburg\n\t\t\t New York,"Aug. 10, 1914","Term expired Aug. 9, 1918.",Paul M. Warburg,New York,"Aug. 10, 1914","Aug. 9, 1918"
3,Frederic A. Delano\n\t\t\t Chicago,"Aug. 10, 1914","Resigned July 21, 1918.",Frederic A. Delano,Chicago,"Aug. 10, 1914","July 21, 1918"
4,W.P.G. Harding\n\t\t\t Atlanta,"Aug. 10, 1914","Term expired Aug. 9, 1922.",W.P.G. Harding,Atlanta,"Aug. 10, 1914","Aug. 9, 1922"
...,...,...,...,...,...,...,...
102,Jeremy C. Stein\n\t\t\t Chicago,"May 30, 2012","Resigned May 28, 2014.",Jeremy C. Stein,Chicago,"May 30, 2012","May 28, 2014"
103,Stanley Fischer\n\t\t\t New York,"May 28, 2014","Resigned Oct. 16, 2017.",Stanley Fischer,New York,"May 28, 2014","Oct. 16, 2017"
104,Lael Brainard\n\t\t\t Richmond,"June 16, 2014","Resigned Feb. 18, 2023.",Lael Brainard,Richmond,"June 16, 2014","Feb. 18, 2023"
105,Randal K. Quarles\n\t\t\t Kansas City,"Oct. 13, 2017","Resigned Dec. 25, 2021.",Randal K. Quarles,Kansas City,"Oct. 13, 2017","Dec. 25, 2021"


In [19]:
reappointed

,Name and District,Date of initial oath of office,Other dates and information relating to membership,Name,District,TermBeginDate,ReappointedDate
0,Janet L. Yellen\n\t\t\t San Francisco,"Aug. 12, 1994","Resigned Feb. 17, 1997; reappointed Oct. 4, 20...",Janet L. Yellen,San Francisco,"Aug. 12, 1994","Oct. 4, 2010"
1,Ben S. Bernanke\n\t\t\t Atlanta,"Aug. 5, 2002","Resigned June 21, 2005; reappointed Feb. 1, 20...",Ben S. Bernanke,Atlanta,"Aug. 5, 2002","Feb. 1, 2006"
2,Jerome H. Powell\n\t\t\t Philadelphia,"May 25, 2012","Reappointed June 16, 2014.",Jerome H. Powell,Philadelphia,"May 25, 2012","June 16, 2014"
3,Michelle W. Bowman\n\t\t\t St. Louis,"Nov. 26, 2018","Reappointed Jan. 30, 2020.",Michelle W. Bowman,St. Louis,"Nov. 26, 2018","Jan. 30, 2020"
4,Lisa D. Cook\n\t\t\t Atlanta,"May 23, 2022","Reappointed Sept. 13, 2023.",Lisa D. Cook,Atlanta,"May 23, 2022","Sept. 13, 2023"


#### 理事的任期表

<!-- name, begin_date, resign_date, district -->

In [20]:
governor_term_info

{'W.G. McAdoo': {'resign_dates': ['Dec. 15, 1918'], 'reappoint_dates': []},
 'John Skelton Williams': {'resign_dates': ['Mar. 2, 1921'],
  'reappoint_dates': []},
 'Charles S. Hamlin': {'resign_dates': [], 'reappoint_dates': []},
 'Paul M. Warburg': {'resign_dates': ['Aug. 9, 1918'], 'reappoint_dates': []},
 'Frederic A. Delano': {'resign_dates': ['July 21, 1918'],
  'reappoint_dates': []},
 'W.P.G. Harding': {'resign_dates': ['Aug. 9, 1922'], 'reappoint_dates': []},
 'Adolph C. Miller': {'resign_dates': ['Feb.\xa03,\xa01936'],
  'reappoint_dates': []},
 'Albert Strauss': {'resign_dates': ['Mar. 15, 1920'], 'reappoint_dates': []},
 'Carter Glass': {'resign_dates': ['Feb. 1, 1920'], 'reappoint_dates': []},
 'Henry A. Moehlenpah': {'resign_dates': ['Aug. 9, 1920'],
  'reappoint_dates': []},
 'David F. Houston': {'resign_dates': ['Mar. 3, 1921'], 'reappoint_dates': []},
 'Edmund Platt': {'resign_dates': ['Sept. 14, 1930'], 'reappoint_dates': []},
 'David C. Wills': {'resign_dates': ['Mar.

In [73]:
governor_terms = governors[["Name", "District", "TermBeginDate"]].copy(deep=True)
governor_terms['TermEndDate'] = [None]*governor_terms.shape[0]
governor_terms['Officier'] = 'Governor'

# 根据辞任表更新理事们的任期结束时间
# 如果有辞任表，则更新其辞任时间；重新任命的职务要手动调整。因此，此处直接使用第一个辞任时间。
for person, term_info in governor_term_info.items():
    # 用第一个辞任时间更新表的辞任时间
    if len(term_info["resign_dates"])!=0:
        governor_terms.loc[governor_terms["Name"] == person, "TermEndDate"] = term_info["resign_dates"][0]

# 根据重任命表，手动维护若干人的信息
governor_terms.loc[governor_terms["Name"] == "Charles S. Hamlin", "TermEndDate"] = (
    "Feb. 1, 1936"
)
reappoint_terms = pd.DataFrame(
    data=[
        ["Janet L. Yellen", "San Francisco", "Feb. 1, 1936", "Feb. 1, 1936", "Governor"],
    ],
    columns=governor_terms.columns
)
# 其他人均填为当前日期
# governor_terms["TermEndDate"] = governor_terms["TermEndDate"].fillna(today)
governor_terms

,Name,District,TermBeginDate,TermEndDate,Officier
0,W.G. McAdoo,Secretary of the Treasury,"Dec. 23, 1913","Dec. 15, 1918",Governor
1,John Skelton Williams,Comptroller of the Currency,"Feb. 2, 1914","Mar. 2, 1921",Governor
2,Charles S. Hamlin,Boston,"Aug. 10, 1914","Feb. 1, 1936",Governor
3,Paul M. Warburg,New York,"Aug. 10, 1914","Aug. 9, 1918",Governor
4,Frederic A. Delano,Chicago,"Aug. 10, 1914","July 21, 1918",Governor
...,...,...,...,...,...
108,Christopher J. Waller,Minneapolis,"Dec. 18, 2020",None,Governor
109,Lisa D. Cook,Atlanta,"May 23, 2022",None,Governor
110,Philip N. Jefferson,New York,"May 23, 2022",None,Governor
111,Michael S. Barr,Chicago,"July 19, 2022",None,Governor


## 最终的理事会成员任期表

### 手工处理的重任命(reappoint)的任期表

In [98]:
reappoint_terms = pd.read_excel('../data/bog/reappoint_terms.xlsx')
reappoint_terms = reappoint_terms.replace(np.nan, None)
reappoint_terms

,Name,District,TermBeginDate,TermEndDate,Officier
0,Lisa D. Cook,Atlanta,"May. 23, 2022","Sept. 13, 2023",Governor
1,Lisa D. Cook,Atlanta,"Sept. 13, 2023",None,Governor
2,Michelle W. Bowman,St. Louis,"Nov. 26, 2018","Jan. 30, 2020",Governor
3,Michelle W. Bowman,St. Louis,"Jan. 30, 2020",None,Governor
4,Jerome H. Powell,Philadelphia,"May. 25, 2012","Jun. 16, 2014",Governor
5,Jerome H. Powell,Philadelphia,"Jun. 16, 2014","Feb. 5, 2018",Governor
6,Jerome H. Powell,Philadelphia,"Feb. 5, 2018","May. 23, 2022",Chair
7,Jerome H. Powell,Philadelphia,"May. 23, 2022",None,Chair
8,Ben S. Bernanke,Atlanta,"Aug. 5, 2002","Jun. 21, 2005",Governor
9,Ben S. Bernanke,Atlanta,"Feb. 1, 2006","Jan. 31, 2010",Chair


In [99]:
bog_terms = pd.concat(
    [
        # governor_terms.loc[
        #     ~governor_terms["Name"].isin(reappoint_terms["Name"].unique().tolist())
        # ],
        # reappoint_terms,
        governor_terms,
        chairs_terms,
        vice_chairs_terms,
        supervision_vice_chairs_terms,
    ],  #
    axis=0,
)
bog_terms = bog_terms.drop_duplicates()
# 填充District
bog_terms["District"] = bog_terms.groupby("Name")["District"].ffill().bfill()

# # 转换格式
bog_terms["TermBeginDate"] = bog_terms["TermBeginDate"].transform(lambda x: parse_datestring(x))
bog_terms["TermEndDate"] = bog_terms["TermEndDate"].transform(lambda x: parse_datestring(x))
bog_terms = bog_terms.sort_values(["Name", "TermBeginDate", "TermEndDate"]).reset_index(drop=True)
# 保存
bog_terms.to_csv("../data/bog/bog_governors_terms.csv", index=False)
bog_terms.info()
bog_terms

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           143 non-null    object        
 1   District       143 non-null    object        
 2   TermBeginDate  143 non-null    datetime64[ns]
 3   TermEndDate    133 non-null    datetime64[ns]
 4   Officier       143 non-null    object        
dtypes: datetime64[ns](2), object(3)
memory usage: 5.7+ KB


,Name,District,TermBeginDate,TermEndDate,Officier
0,"A.L. Mills, Jr.",San Francisco,1952-02-18,1965-02-28,Governor
1,Adolph C. Miller,San Francisco,1914-08-10,1936-02-03,Governor
2,Adriana D. Kugler,Richmond,2023-09-13,NaT,Governor
3,Alan Greenspan,New York,1987-08-11,2006-01-31,Governor
4,Alan Greenspan,New York,1987-08-11,2006-01-31,Chair
...,...,...,...,...,...
138,Wayne D. Angell,Kansas City,1986-02-07,1994-02-09,Governor
139,William H. Woodin,Secretary of the Treasury,1933-03-04,1933-12-31,Governor
140,William W. Sherrill,Dallas,1967-05-01,1971-11-15,Governor
141,"Wm. McC. Martin, Jr",New York,1951-04-02,1970-01-31,Chair


In [100]:
# bog_terms.iloc[0:100, 3].transform(parse_datestring)

In [101]:
bog_terms.loc[pd.isna(bog_terms['TermEndDate'])].sort_values(by="Name")

,Name,District,TermBeginDate,TermEndDate,Officier
2,Adriana D. Kugler,Richmond,2023-09-13,NaT,Governor
22,Christopher J. Waller,Minneapolis,2020-12-18,NaT,Governor
71,Jerome H. Powell,Philadelphia,2012-05-25,NaT,Governor
72,Jerome H. Powell,Philadelphia,2018-02-05,NaT,Chair
86,Lisa D. Cook,Atlanta,2022-05-23,NaT,Governor
95,Michael S. Barr,Chicago,2022-07-19,NaT,Governor
96,Michael S. Barr,Chicago,2022-07-19,NaT,Vice Chair for Supervision
97,Michelle W. Bowman,St. Louis,2018-11-26,NaT,Governor
108,Philip N. Jefferson,New York,2022-05-23,NaT,Governor
109,Philip N. Jefferson,New York,2023-09-13,NaT,Vice Chair


In [105]:
# 鲍曼任期查询
# print(bog_terms.query("Name=='Michelle W. Bowman'"))
# print(bog_terms.query("Name=='Jerome H. Powell'"))
bog_terms.loc[
    bog_terms["Name"].isin(
        ["Janet L. Yellen", "Jerome H. Powell", "Ben S. Bernanke", "Michelle W. Bowman"]
    )
].sort_values(by='Name')

,Name,District,TermBeginDate,TermEndDate,Officier
14,Ben S. Bernanke,Atlanta,2002-08-05,2005-06-21,Governor
15,Ben S. Bernanke,Atlanta,2006-02-01,2014-01-31,Chair
66,Janet L. Yellen,San Francisco,1994-08-12,1997-02-17,Governor
67,Janet L. Yellen,San Francisco,2010-10-04,2014-02-03,Vice Chair
68,Janet L. Yellen,San Francisco,2014-02-03,2018-02-03,Chair
71,Jerome H. Powell,Philadelphia,2012-05-25,NaT,Governor
72,Jerome H. Powell,Philadelphia,2018-02-05,NaT,Chair
97,Michelle W. Bowman,St. Louis,2018-11-26,NaT,Governor


In [106]:
bog_terms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           143 non-null    object        
 1   District       143 non-null    object        
 2   TermBeginDate  143 non-null    datetime64[ns]
 3   TermEndDate    133 non-null    datetime64[ns]
 4   Officier       143 non-null    object        
dtypes: datetime64[ns](2), object(3)
memory usage: 5.7+ KB


In [116]:
def query_incumbent_governors(date: str, bog_terms: pd.DataFrame):
    # 转为日期格式
    date = parse_datestring(date)
    # 筛选出在任任期
    incumbent = bog_terms.loc[
        (bog_terms["TermBeginDate"] <= date)
        & ((bog_terms["TermEndDate"] >= date) | (pd.isna(bog_terms["TermEndDate"])))
    ].reset_index(drop=True)
    # 筛选出最高职位
    _chair_mapping = {"Chair": 0, "Vice Chair": 1, "Vice Chair for Supervision": 2, "Governor": 3}
    incumbent["Position"] = incumbent["Officier"].map(_chair_mapping)
    incumbent = incumbent.sort_values(
        ["Position", "Name", "TermBeginDate"], ascending=[True, True, False]
    )
    incumbent = incumbent.drop_duplicates(subset=["Name"], keep="first").drop(columns=['Position']).reset_index(drop=True)
    return incumbent

# query_incumbent_governors('Feb. 2, 2020', bog_terms)
query_incumbent_governors("Aug. 25, 2023", bog_terms) #['Name'].unique().tolist()

,Name,District,TermBeginDate,TermEndDate,Officier
0,Jerome H. Powell,Philadelphia,2018-02-05,NaT,Chair
1,Michael S. Barr,Chicago,2022-07-19,NaT,Vice Chair for Supervision
2,Christopher J. Waller,Minneapolis,2020-12-18,NaT,Governor
3,Lisa D. Cook,Atlanta,2022-05-23,NaT,Governor
4,Michelle W. Bowman,St. Louis,2018-11-26,NaT,Governor
5,Philip N. Jefferson,New York,2022-05-23,NaT,Governor
